In [3]:
!pwd

'pwd' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [4]:
import pandas as pd

# Carregar o arquivo Excel enviado
file_path = 'Pixsee Planet subscription .xlsx'
df = pd.read_excel(file_path)

# Verificar as primeiras linhas do DataFrame para entender a estrutura
df.head()


,Subscription plan,Standard,Gold Star
0,Cost,Complimentary,Approx.$8USD/mo
1,Cloud capacity,Capacity limit 1G,Capacity limit 30G
2,Digital Keepsakes,NaN,NaN
3,Manual Capture (5MP) and Record ⓘ,✓,✓
4,Camera Timer ⓘ,✓,✓


In [9]:
import json


# Process the data with improved categorization
structured_data_with_categories = []
current_subgroup = None  # Track the current subgroup

# Add general details (first row)
for plan, cost in zip(df.columns[1:], df.iloc[0, 1:]):
    structured_data_with_categories.append({
        "id": f"{plan.lower()}_general",
        "type": "general_info",
        "plan": plan,
        "category": "Subscription Plan",  # Add explicit category
        "feature": "Cost",
        "details": cost,
        "subgroup": None
    })

# Process each row to associate with subgroups and categorize features
for index, row in df.iterrows():
    if index <= 1:
        # Skip the first two rows as the first row is already processed
        continue

    feature = row[df.columns[0]]  # The first column contains subgroups or features

    # Check if the row represents a new subgroup
    if pd.isna(row[df.columns[1]]) and pd.isna(row[df.columns[2]]):
        # If both plan columns are NaN, this row is a subgroup
        current_subgroup = feature
        continue

    # Process the feature details under the current subgroup
    for plan in df.columns[1:]:
        if pd.notna(row[plan]):
            structured_data_with_categories.append({
                "id": f"{plan.lower()}_{index}",
                "type": "feature",
                "plan": plan,
                "category": "Feature",  # Add explicit category
                "feature": feature,
                "details": row[plan],
                "subgroup": current_subgroup
            })

# Save the structured data with categories to a JSON file
output_path_with_categories = 'structured_subscription_data_with_categories.json'
with open(output_path_with_categories, 'w') as f:
    json.dump(structured_data_with_categories, f, indent=4)

output_path_with_categories


'structured_subscription_data_with_categories.json'